## Introdução

Word2Vec é um algoritmo bem popular usado para gerar representações vetoriais densas de palavras em grandes textos usando aprendizado não supervisionado. Essas representações são úteis para muitas tarefas de processamento de linguagem natural (PLN/NLP), como análise de sentimentos, reconhecimento de entidades nomeadas e tradução automática.


Modelos populares que aprendem tais representações ignoram a morfologia das palavras, atribuindo um vetor distinto a cada palavra. Essa é uma limitação, especialmente para idiomas com vocabulários grandes e muitas palavras incomuns. O algoritmo *SageMaker BlazingText* pode aprender representações vetoriais associadas aos caracteres n-gramas; representando palavras como a soma dessas representações de n-gramas de caracteres [1]. Esse método permite que o *BlazingText* gere vetores para palavras fora do vocabulário (out-of-vocabulary / OOV), conforme demonstrado neste notebook.

Ferramentas populares como [FastText](https://github.com/facebookresearch/fastText) aprendem a incorporação de subpalavras para gerar representações, mas têm uma escala ruim, pois podem ser executadas apenas em CPUs. O BlazingText estende o modelo FastText para alavancar GPUs, fornecendo mais de 10x aceleração, dependendo do hardware.

[1] P. Bojanowski, E. Grave, A. Joulin, T. Mikolov, [Enriching Word Vectors with Subword Information](https://arxiv.org/pdf/1607.04606.pdf)

## Configurações


In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

In [ ]:
sess = sagemaker.Session()

role = get_execution_role()
print(role) # Pega a Role/Função que criamos e que o Notetbook está rodando

bucket = sess.default_bucket() # Pega o bucket padrão da Sessão (ou coloque aqui teu bucket já usado)
print(bucket)
prefix = 'blazingtext/aulasagemaker' #Pasta em que todo o trabalho será executado

### Leitura dos dados para treinamento

O BlazingText espera um único arquivo de texto pré-processado com tokens separados por espaço e cada linha do arquivo deve conter uma única frase. Eu já deixei o arquivo 

Neste exemplo, vamos treinar os vetores no conjunto de dados [text8](http://mattmahoney.net/dc/textdata.html) (100 MB), que é uma versão pequena (já pré-processada) do dump da Wikipedia em inglês. O arquivo já está pronto na pasta `datasets/textmining/text8.gz` e precisa ser descompactado.

In [ ]:
# Uncompressing
!gzip -d datasets/textmining/text8.gz -f

Após a descompactar o arquivo, precisamos carregá-los no S3 para que possam ser consumidos pelo SageMaker para o treinamento. Usaremos o Python SDK para fazer upload desse arquivo no bucket e na pasta escolhidos acima.

Execute esta célula e confira se o arquivo está no local esperado.

In [ ]:
train_channel = prefix + '/train'

sess.upload_data(path='datasets/textmining/text8', bucket=bucket, key_prefix=train_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)

Então, precisamos configurar um local de saída no S3, onde o arquivo do modelo será salvo. Importante: é aí que ficará o arquivo resultante do treinamento do modelo.

In [ ]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
print(s3_output_location)

## Preparando o treino  

Agora que terminamos todas as configurações necessárias, estamos prontos para treinar nosso detector de objetos. Para começar, vamos criar um objeto ``sageMaker.estimator.Estimator``. Este estimador iniciará o trabalho de treinamento.

In [ ]:
region_name = boto3.Session().region_name
print(region_name)

In [ ]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

## Treinando o modelo BlazingText para gerar vetores de palavras

In [ ]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.2xlarge', # Use of ml.p3.2xlarge is highly recommended for highest speed and cost efficiency
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Ajuste dos hyperparâmetros.  
Consulte a [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) para a lista completa.

In [ ]:
bt_model.set_hyperparameters(mode="skipgram",
                             epochs=5,
                             min_count=5,
                             sampling_threshold=0.0001,
                             learning_rate=0.05,
                             window_size=5,
                             vector_dim=100,
                             negative_samples=5,
                             subwords=True, # Enables learning of subword embeddings for OOV word vector generation
                             min_char=3, # min length of char ngrams
                             max_char=6, # max length of char ngrams
                             batch_size=11, #  = (2*window_size + 1) (Preferred. Used only if mode is batch_skipgram)
                             evaluation=True)# Perform similarity evaluation on WS-353 dataset at the end of training

Prepara variável que possui os dados que serão usados no treinamento.

In [ ]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data}

Depois que o trabalho terminar, uma mensagem "Trabalho concluído" será impressa. O modelo treinado pode ser encontrado no bucket S3 que foi configurado como `output_path` no estimador.

Este processo deve demorar de **7 a 10 minutos** ... podemos acompanhar  pelo portal também 😊

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

## Hosting / Endpoint  

Após a conclusão do treinamento, podemos implantar o modelo treinado como um endpoint hospedado em realtime do Amazon SageMaker. Isso nos permitirá fazer predições (ou scoring/inferência) a partir do modelo. Observe que não precisamos hospedar no mesmo tipo de instância que costumávamos treinar. Como os pontos de extremidade da instância estarão em funcionamento por muito tempo, podemoos escolher uma instância mais barata para inferência.

Este processo deve demorar de **4 a 8 minutos**... podemos acompanhar 😊 pelo portal também :-)

In [ ]:
bt_endpoint = bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.m4.xlarge')
print("Concluído!")
print("EndpointName do modelo: {0}".format( bt_endpoint.endpoint) )

### Obtendo representações vetoriais para palavras [incluindo out-of-vocabulary (OOV) words]

Como treinamos com o parâmetro ** ```subwords ="True"``` **, podemos obter representações vetoriais para qualquer palavra - incluindo palavras com erros ortográficos ou palavras que não estavam presentes no conjunto de dados de treinamento.
Se treinarmos sem o sinalizador de subpalavras, o treinamento será muito mais rápido, mas o modelo não poderá gerar vetores para palavras OOV. Em vez disso, retornará um vetor de zeros para essas palavras.

#### Use o padrão JSON para predição  

O payload deve conter uma lista de palavras com a chave como "**instances**". O AWS Sagemaker suporta o tipo de conteúdo `application/json`.

In [ ]:
words = ["awesome", "awweeesome"]

payload = {"instances" : words}

response = bt_endpoint.predict(json.dumps(payload))

vecs = json.loads(response)
print(vecs)

In [ ]:
print( len(vecs) )
print( len(vecs[0]['vector']) )
print( len(vecs[1]['vector']) )

Como esperado, obtemos um vetor n-dimensional (onde n é vector_dim, conforme especificado nos hiperparâmetros) para cada uma das palavras.

---

Ok! Agora é avaliar se está funcionando a partir de outras chamadas.